PINN Introduction
=================

In [ ]:
# Run this cell is you are using Google Colab
!git clone https://github.com/CPSHub/LecturePhysicsAwareML.git
%cd LecturePhysicsAwareML/PINNs
%pip install -e .

In [ ]:
# Run this cell, if you are working locally
%cd ..
%pip install -e .

### Define boundary conditions.

In [1]:
import jax.numpy as jnp
from paml_pinns.nlebb import Config, get_data_decorator


@get_data_decorator
def get_data(config: Config):
    if config.bc_case == 0:
        u_bc_coords = jnp.array([0.0])
        u_bc_values = jnp.array([0.0])
        w_bc_coords = jnp.array([0.0])
        w_bc_values = jnp.array([0.0])
        w_x_bc_coords = jnp.array([0.0])
        w_x_bc_values = jnp.array([0.0])
        N_bc_coords = jnp.array([config.L])
        N_bc_values = jnp.array([0.0])
        M_bc_coords = jnp.array([config.L])
        M_bc_values = jnp.array([0.0])
        Q_bc_coords = jnp.array([config.L])
        Q_bc_values = jnp.array([config.F])
    else:
        NotImplementedError(
            "Data generation is not implemented for these boundary conditions."
        )

    bc = {
        "u_bc_coords": u_bc_coords,
        "u_bc_values": u_bc_values,
        "w_bc_coords": w_bc_coords,
        "w_bc_values": w_bc_values,
        "w_x_bc_coords": w_x_bc_coords,
        "w_x_bc_values": w_x_bc_values,
        "N_bc_coords": N_bc_coords,
        "N_bc_values": N_bc_values,
        "M_bc_coords": M_bc_coords,
        "M_bc_values": M_bc_values,
        "Q_bc_coords": Q_bc_coords,
        "Q_bc_values": Q_bc_values,
    }
    return bc


### Configure beam properties.

In [2]:
from paml_pinns.nlebb import get_config_decorator
from typing import Tuple


@get_config_decorator
def get_config(bc_case: int) -> Tuple[float, float, float, float, float, float]:
    if bc_case == 0:
        E = 1.0
        A = 1.0
        II = 1.0
        L = 1.0
        F = 1.0
        f = 0.0
        q = 0.0
    else:
        NotImplementedError(
            "No configuration implemented for these boundary conditions."
        )

    return E, A, II, L, F, f, q

### Data generation and model creation

In [ ]:
import jax
from paml_pinns.nlebb import create_pinn


config = get_config(bc_case=0)
x, y, bc, params = get_data(config)

key = jax.random.PRNGKey(1234)
model = create_pinn(params, bc, key=key, mixed_formulation=False)
weights = {"u": 1.0, "w": 1.0, "w_x": 1.0, "N": 1.0, "M": 1.0, "Q": 1.0}
params

### Calibration and Evaluation

In [ ]:
from paml_pinns import train
from paml_pinns.nlebb import evaluate

model = train(model, x, weights, steps=50_000, batch_size=128)
evaluate(model, x, y)

Tasks
=====

#### Task 1: Mixed formulation

For `bc_case=0`, ...

a) Set `mixed_formulation=False`. Can the model approximate the solution?

b) Now set `mixed_formulation=True`, while keeping all other parameters the same. Can the model approximate the solution now? How can the observed differences be explained?

c) Next select your preferred architecture and increase the value of `E`. What do you observe? How could the effects be mitigated?

#### Task 2: Discussion

Based on your experience from the the PINN task, collect some advantages and disadvantages of PINNs over traditional numerical methods like FEM. When would you prefer to use PINNs, when would you obt for FEM? Discuss your thoughts with a partner.

#### Task 3 (Optional): Implementation of different boundary conditions

Calibrate the PINN to different boundary conditions by modifying the `get_data`
function and the `paml_pinn.nlebb.cases` module. Can you find an analytical soulution to validate your implementation?